<a href="https://colab.research.google.com/github/vifirsanova/XLNet-Text-Generation/blob/main/XLNet_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Туториал подготовила [Виктория Фирсанова](https://github.com/vifirsanova)

<td>
<a href="https://github.com/vifirsanova/XLNet-Text-Generation"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/github_32px.png"/>Источник на GitHub</a>
</td>


# Генерация текста с XLNet

**Q**: Почему XLNet?

**A**: GPT отлично справляется с генерацией текстов (предсказанием следующего токена для заданной последовательности), но это однононаправленная модель. XLNet позволяет генерировать текст с помощью двунаправленной архитектуры (сравните с BERT). 

Для построения модели мы воспользуется [XLNet от HuggingFace](https://huggingface.co/transformers/model_doc/xlnet.html).




# Загрузка модулей

Не забудьте выбрать GPU как среду выполнения: 

1. Среда выполнения
2. Сменить среду выполнения
3. Аппаратный ускоритель
4. GPU
5. Сохранить

Готово! Вы восхитительны. Посмотрим, что в нашем распоряжении в ячейке ниже.

In [ ]:
!nvidia-smi

Sun Dec 13 16:51:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Загрузим [HuggingFace Transformers](https://huggingface.co/transformers/)



In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.3MB/s 
     |████████████████████████████████| 2.9MB 54.6MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b47018fa1daf1d1fdee46884ce743cc10a775ff7777cc44b1e2731cf05d5f712
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Загрузим датасет

Здесь мы будем использовать [CMU Book Summary Dataset](http://www.cs.cmu.edu/~dbamman/booksummaries.html).

Загрузим датасет напрямую по ссылке.

In [ ]:
!wget http://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz

--2020-12-13 16:51:44--  http://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16795330 (16M) [application/x-gzip]
Saving to: ‘booksummaries.tar.gz’

booksummaries.tar.g 100%[===================>]  16.02M   923KB/s    in 18s     

2020-12-13 16:52:02 (900 KB/s) - ‘booksummaries.tar.gz’ saved [16795330/16795330]



Извлечем данные и посмотрим на них

In [ ]:
import pandas as pd
df = pd.read_csv('/content/booksummaries.tar.gz', sep='\t', names=['wiki_id', 'freebase_id', 'title', 'author', 'date', 'genres', 'summary'])[16:-1]
df.sample(10)

,wiki_id,freebase_id,title,author,date,genres,summary
5071,3884501,/m/0b4_x1,Damaged Goods,Russell T. Davies,NaN,"{""/m/014dfn"": ""Speculative fiction""}","The novel is set in Britain in 1987, and invo..."
11161,14298413,/m/03c_sgm,Ruth Hall,Fanny Fern,1854,"{""/m/016lj8"": ""Roman \u00e0 clef""}",The autobiographical novel can be divided int...
3909,2678701,/m/07xdpz,The Loveday Fortunes,Kate Tremayne,2000-12-07,"{""/m/01qxvh"": ""Romance novel"", ""/m/0hwxm"": ""Hi...",When the Lovedays' banker is found dead in th...
9266,9764872,/m/02prhkj,The Ugly Swans,Arkady Strugatsky,NaN,"{""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""...",The action takes place in an uncertain mildly...
2175,1151016,/m/04bnyz,The Basic Eight,Daniel Handler,1998-04,"{""/m/02xlf"": ""Fiction""}",Flannery Culp is a senior at Roewer High Scho...
4507,3290351,/m/093mkr,Topdog/Underdog,Suzan-Lori Parks,NaN,NaN,The play chronicles the adult lives of two Af...
15134,26839511,/m/0c020xz,The Thousand Autumns of Jacob de Zoet,David Mitchell,2010-05-13,"{""/m/0488wh"": ""Literary fiction""}",The novel begins in the summer of 1799 at the...
14426,24021398,/m/07k5tyc,Vampire Zero,David Wellington,2008,NaN,After Pennsylvania State Trooper and vampire ...
13020,19723244,/m/04n2gch,Minnie and Moo: The Night of the Living Bed,NaN,NaN,NaN,After having a nightmare about a giant mouse ...
16449,33912418,/m/0hnc3g8,Durgeshnandini,Bankim Chandra Chattopadhyay,1865,"{""/m/05hgj"": ""Novel""}",The story is set in the backdrop of Pathan-Mu...


Теперь можно поделить датасет на три выборки: обучающую, проверочную и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(df, train_size = 0.8, random_state = 1)
test, val = train_test_split(temp, train_size = 0.5, random_state = 1)

Что получилось?

In [ ]:
print(f'Длина обучающей выборки: {len(train)} \nДлина проверочной выборки: {len(val)} \nДлина тестовой выборки: {len(test)}')

Длина обучающей выборки: 13247 
Длина проверочной выборки: 1656 
Длина тестовой выборки: 1656


Обработаем датасет

In [ ]:
import re 

def dataset(set, path):
    texts = df['summary'].tolist() # получаем список текстов
    data = ''
    f = open(path, 'w')

    for text in texts:
        
        text = str(text).strip() # удаляем лишние пробелы
        text = re.sub(r"\s", " ", text) # заменяем на пробел соответствующее регулярное выражение
        start = '<s> ' # добавляем маркеры для начала и конца предложения
        end = ' </s>'
        data += start + text + end + '\n' # собираем 
        
    f.write(data)
    return data

Соберем три выборки

In [ ]:
train_set = dataset(train, 'train.txt')
val_set = dataset(val, 'val.txt')
test_set = dataset(test, 'test.txt')

Что получилось? 

In [ ]:
test_set[:100]

'<s> Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he'

# Токенизация + векторизация

Загрузим токенизатор для XLNet из HuggingFace Transformers.

Мы будем использовать предобученную модель [XLNet base cased](https://huggingface.co/xlnet-base-cased)


In [ ]:
from transformers import XLNetTokenizerFast
tokenizer = XLNetTokenizerFast.from_pretrained('xlnet-base-cased')

Посмотрим, как работает быстрый токенизатор 

In [ ]:
tokenizer('Hi! Have a nice day 👍')

{'input_ids': [4036, 136, 4386, 24, 2101, 191, 17, 0, 4, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

"Упакуем" наши сеты для обучения сети

In [ ]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, val_path, test_path, tokenizer):
    train_dataset = LineByLineTextDataset(
          tokenizer = tokenizer,
          file_path = train_path,
          block_size = 128)
     
    val_dataset = LineByLineTextDataset(
          tokenizer = tokenizer,
          file_path = val_path,
          block_size = 128)

    test_dataset = LineByLineTextDataset(
          tokenizer = tokenizer,
          file_path = test_path,
          block_size = 128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer, mlm = False)

    return train_dataset, val_dataset, test_dataset, data_collator

train_dataset, val_dataset, test_dataset, data_collator = load_dataset('train.txt', 'val.txt', 'test.txt', tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


# Обучение НС

Аргументы для обучения модели

In [ ]:
from transformers import Trainer, TrainingArguments, XLNetLMHeadModel

model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased")

args = TrainingArguments(
    output_dir = "./xlnet",
    overwrite_output_dir = True,
    num_train_epochs = 1,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64, 
    eval_steps = 400, 
    save_steps = 800, 
    warmup_steps = 500)

trainer = Trainer(
    model = model,
    args = args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = val_dataset)

Запускаем

In [ ]:
trainer.train()

Step,Training Loss
500,0.089155


TrainOutput(global_step=518, training_loss=0.08609223939679764)

Сохраняем

In [ ]:
trainer.save_model()

# Генерация текста

In [ ]:
from transformers import pipeline

text = pipeline('text-generation', model='./xlnet', tokenizer='xlnet-base-cased', config={'max_length': 512})

Пример генерации текста

In [ ]:
text('This is')[0]['generated_text']

'This is a "true" ". Fists and pikewi. The first few few "true" ". The first few of "true" ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", "'